# EASY pilot study downloads
This notebook can be used to download the data associated with the
"EASY Study - 75 Image, full featureset" study on the ISIC archive
available here:

https://isic-archive.com/api/v1/#!/study/study_find

And from there we can see that this study has the (mongodb Object)
ID "58d9189ad831133741859b5d", with which we can access all the
data associated with that study.

## Notes on the ISIC API
Here's a few important notes related to the ISIC API:

- whenever you are logging in using a web browser, make sure the
  login is performed on the actual https://isic-archive.com/
  domain, not the www.isic-archive.com domain, since they do not
  share cookies/headers, meaning that a login at the www. website
  will not carry over! To do so, visit https://isic-archive.com/admin
- when accessing annotation images (masks) or superpixel arrays,
  the feature must be given as a URL mangled name, not as on the
  API documentation specified (featureId)

In [2]:
# username (leave at None if you don't have one)
# username = None
import getpass
username = 'weberj3@mskcc.org'
password = None # do NOT enter here, will be requested via getpass!
if not username is None:
    password = getpass.getpass('Please enter the password for user %s: ' % (username))

# ObjectID for the "EASY Study - 75 Image, full featureset" study
studyId = '5a32cde91165975cf58a469c'
studyName = 'EasyPilot'

# settings
ImageFolder = 'ISICImages'
AnnotationFolder = 'Annotations'

# imports
import os
from ISICApi import ISICApi

# get ISIC API reference
api = ISICApi(None, username, password)

Please enter the password for user weberj3@mskcc.org: ········


In [3]:
# make folders if necessary
if not os.path.exists(ImageFolder):
    os.mkdir(ImageFolder)
if not os.path.exists(AnnotationFolder):
    os.mkdir(AnnotationFolder)

In [4]:
# function for mangling feature names
featureIds = {}
def featureId(feature):
    if feature in featureIds:
        return featureIds[feature]
    letters = [letter if ord(letter) > 64 and ord(letter) < 123 else '%s%02x' % ('%', ord(letter)) for letter in feature]
    idVal = ''.join(letters)
    featureIds[feature] = idVal
    return idVal
featureNames = {}
def featureName(feature):
    if feature in featureNames:
        return featureNames[feature]
    letters = [letter for letter in feature if ord(letter) > 64 and ord(letter) < 123]
    nameVal = ''.join(letters)
    featureNames[feature] = nameVal
    return nameVal

In [5]:
# get study information
studyInfo = api.getJson('study/%s' % (studyId))
studyFeatures = studyInfo['features']
studyImage = studyInfo['images']
studyComplete = studyInfo['userCompletion']

# and users with at least 75 images annotated
studyUsers = list(filter(lambda user: studyComplete[user['_id']] >= 140, studyInfo['users']))
print('Study "%s" has %s annotators with at least 140 images.' % (studyInfo['name'], len(studyUsers)))

Study "ISIC Annotation Study - All Features" has 5 annotators with at least 140 images.


In [6]:
# download ISIC images (if not already in folder)
imageIds = []
for ic in range(len(studyImage)):
    imageIds.append(studyImage[ic]['_id'])
    localFile = '%s/%s.jpg' % (ImageFolder, studyImage[ic]['name'])
    if not os.path.exists(localFile):
        print('Downloading %s...' % (studyImage[ic]['name']))
        api.getFile('image/%s/download' % (studyImage[ic]['_id']), localFile)

In [7]:
# download annotations
for user in studyUsers:
    print('Downloading annotations for user %s %s...' % (user['firstName'], user['lastName']))
    userId = user['_id']
    userName = user['lastName']
    annotations = api.getJsonList('annotation?studyId=%s&userId=%s&state=complete&detail=true' % (studyId, userId))
    for annotation in annotations:
        annotationId = annotation['_id']
        imageName = annotation['image']['name']
        for markup in annotation['markups'].keys():
            fId = featureId(markup)
            fName = featureName(markup)
            localFile = '%s/%s_%s_%s_%s.png' % (AnnotationFolder, studyName, imageName, userName, fName)
            if not os.path.exists(localFile):
                api.getFile('annotation/%s/%s/mask' % (annotationId, fId), localFile)